In [1]:
import pandas as pd
import requests
import json
import csv
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
def get_educ_school_acs_data(base_url, vars_str, fips_df, op_path, op_file, key):
    """
    :param base_url: Different base url for 2000 and 2010 years
    :param fips_df: df with state and county fips codes
    :param op_file: op file name to which the data needs to be written
    :param census_year: year for which decennial census data
    :return:
    """
    
    # take a count variable to help distinguish 1st request from subsequent requests so that we take the variable
    # names only from the 1st call as the header names and skip varibale names from the subsequent calls.
    response = ''
    count = 0
    # open a new file with the required output file name
    with open(f'{op_path}/{op_file}', 'w') as file_wrtr:
        # take a csv writer object to write to this output file
        op_file_writer = csv.writer(file_wrtr)

        # Iterate over each row in the state_county fips df
        for row in fips_df.itertuples():
            print('row: ', row)
            # create the request url for each state and fips code to get the data for all county subdivisions under it
            try:
                """ Use timer if needed """
                if count in [20, 40, 60, 80, 100]:
                    time.sleep(2000) # milliseconds

                st_fips = str(getattr(row, "state"))
                if st_fips.__len__() < 2:
                    st_fips = '0' + st_fips
                    
                # cnty_fips = str(getattr(row, "county"))

                # https://api.census.gov/data/2019/acs/acs1?get=NAME,B01001_001E&for=county%20subdivision:*&in=state:36&key=YOUR_KEY_GOES_HERE
                # url = f'{base_url}/{year}/acs/acs1?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:{cnty_fips}&key=aefa63e9ca460169068f42b9ebc3101db025882f'

                url = f'{base_url}/{year}/acs/acs5?get={vars_str}&for=county%20subdivision:*&in=state:{st_fips}%20county:*&key=key'

                print(f'url at count{count}: ', url)

                headers = requests.utils.default_headers()
                headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36'
                response = requests.get(url, headers=headers)
                print('response: ', response)
                # load the json in to python object which would be list of all entries in the json object
                resp = json.loads(response.content)
                # If this is the first call(count=0), then write the entire content so that the 1st row with the variable names is the header
                if count == 0:
                    # iterate over respone python object and write each row to the csv
                    for res in resp:
                        op_file_writer.writerow(res)
                else:
                    # skipping 1st row which are the variable names starting from 2nd calls.
                    for res in resp[1:]:
                        op_file_writer.writerow(res)

                count += 1
                op_file_writer.writerow([])
                print('Total API Calls: ', count)
            except Exception as ex:
                print('Error code: ', response.status_code)
                print('Error Response: ', response.content)
                print('Exception: ', ex)
                break



# final_df = create_final_twnshp_file(
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/twnshps_api',
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/new_census_townships_90_initial.csv')
#
# # sort the final df by state, county and then county subdivision to make sure they are in required ascending order. Default is ascending
# final_df_sorted = final_df.sort_values(by=['state', 'county', 'county subdivision'])
#
# # write the final df to a csv
# final_df_sorted.to_csv(
#     '/Users/salma/Studies/Research/Criminal_Justice/research_projects/US Crime Analytics/data/census_cities_1990/new_census_townships_90.csv',
#     index=False)

In [17]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

st_cnty_fips_00 = pd.read_csv('/Users/salma/Library/CloudStorage/OneDrive-UniversityofToledo/Research/dissertation_work/education-schools-crime/data/st_cnty_fips_2000.csv')

for year in years:
    get_educ_school_acs_data(base_url='https://api.census.gov/data', 
                             vars_str='COUNTY, COUSUB, PLACE, STATE, TRACT, B14001_002E, B14001_006E, B14001_007E, B14001_008E, B14001_009E, B14002_003E, B14002_013E, B14002_016E, B14002_019E, B14002_022E, B14002_027E, B14002_037E, B14002_040E, B14002_043E, B14002_046E, B06009_001E, B06009_002E, B06009_003E, B06009_004E, B06009_005E, B06009_006E, B15001_001E, B15001_002E, B15001_003E, B15001_011E, B15001_043E, B15001_044E, B15001_052E, B15002_001E, B15002_002E, B15002_019E, C15002A_001E, C15002A_002E, C15002A_007E, C15002B_001E, C15002B_002E, C15002B_007E, C15002H_001E, C15002H_002E, C15002H_007E, C15002I_001E, C15002I_002E, C15002I_007E', 
                             fips_df=st_cnty_fips_00,
                             key='aefa63e9ca460169068f42b9ebc3101db025882f'
                             op_path='/Users/salma/Research/dissertation_work/data/educ_schl_acs',
                             op_file=f'educ_school_acs_initial_{year}.csv')

row:  Pandas(Index=0, state=1, county=1)
url at count0:  https://api.census.gov/data/2009/acs/acs5?get=COUNTY, COUSUB, PLACE, STATE, TRACT, B14001_002E, B14001_006E, B14001_007E, B14001_008E, B14001_009E, B14002_003E, B14002_013E, B14002_016E, B14002_019E, B14002_022E, B14002_027E, B14002_037E, B14002_040E, B14002_043E, B14002_046E, B06009_001E, B06009_002E, B06009_003E, B06009_004E, B06009_005E, B06009_006E, B15001_001E, B15001_002E, B15001_003E, B15001_011E, B15001_043E, B15001_044E, B15001_052E, B15002_001E, B15002_002E, B15002_019E, C15002A_001E, C15002A_002E, C15002A_007E, C15002B_001E, C15002B_002E, C15002B_007E, C15002H_001E, C15002H_002E, C15002H_007E, C15002I_001E, C15002I_002E, C15002I_007E&for=county%20subdivision:*&in=state:01%20county:*&key=aefa63e9ca460169068f42b9ebc3101db025882f
response:  <Response [400]>
Error code:  400
Error Response:  b"error: error: 'get' is limited to 50 variables"
Exception:  Expecting value: line 1 column 1 (char 0)
row:  Pandas(Index=0, state=1